# Outline
1. [get samp names](#samps)<br>
1. [map samp names to aug, leftover, combo, redo, plate 7](#map)<br>
1. [get genotype calls](#geno calls)<br>
1. [print results to table](#print)<br>
1. [calc mistying and dropouts](#calcs)<br>
1. [make moran infile](#infile)

In [ ]:
imp = open('/home/lindb/pythonimports.py').read()
exec(imp)
from __future__ import division

from scipy.special import comb
from tabulate import tabulate
%matplotlib inline 

In [ ]:
# first run needed to be compared between regenotyping; fix errors
# src = '/home/lindb/teakettle/genemarker/outfiles/' 
# dst = '/home/lindb/teakettle/genemarker/outfiles_first/'
# shutil.move(src,dst)
# if not op.exists(src):
#     mkdir(src)

<a id="samps"></a>
# Get samp names

Determine which samps were regenotyped, and the plates that they're in. By the end of this I would like to have a list of plates that have samps that were genotyped more than once

In [ ]:
#get tree location data from master data 
t = pd.read_csv('/home/lindb/teakettle/data/trees.txt',sep='\t',index_col=None)
t.replace("TRUE",True,inplace=True)
t.replace("FALSE",False,inplace=True)
t.replace('True',True,inplace=True)
t.replace('False',False,inplace=True)
t = t.loc[t['sampled'] != False,:].copy()
t.index = np.arange(len(t.index))
t.head()

In [ ]:
def convert(s):
    num,plot = (s[:-3],s[-3:])
    if 'new' in num:
        num,new = (num[:-3],num[-3:])
        while len(num) < 3:
            num = '0%s' % num
    else:
        new = ""
    samp = "%s_%s%s" % (plot,new,num)
    return(samp)
t['converted'] = t['COMBO'].apply(lambda x: convert(x))
filE = '/home/lindb/teakettle/data/trees_with_converted.txt'
t.to_csv(filE, header=True,index=False,sep='\t')

In [ ]:
'COMBO' in t

In [ ]:
'converted' in t

In [ ]:
'11un1' in t['COMBO'].tolist()

In [ ]:
'un1_11' in t['converted'].tolist()

<a id="map"></a>
# map samp names to aug, leftover, combo, redo, plate 7

In [ ]:
# make dictionary to fill in samples for combined, redo, aug, and leftover plates
# mfiles contain the well ID and samp name to be used as a map for Cornell outfiles
DIR = '/home/lindb/teakettle/genemarker/map/locfirst/'
mfiles = [f for f in fs(DIR) if '_fam' not in f and '_ned' not in f]
locfirst = [f for f in fs(DIR) if '_fam' not in f and '_ned' not in f]
DIR = '/home/lindb/teakettle/genemarker/map/loclast/'
[mfiles.append(f) for f in fs(DIR) if '_fam' not in f and '_ned' not in f]
loclast = [f for f in fs(DIR) if '_fam' not in f and '_ned' not in f]
len(mfiles)

In [ ]:
# known samps that won't be in t['converted'], most are Jeffrey pine that were accidentally included
skips = ['uc2_3141',
         'uc2_5185',
         'uc2_9554',
         'uc2_9592',
         'un1_2749',
         'un2_3126',
         'un3_2588',
         'us3_1752',
         'un1_5587',
         'bc1_7932',
         'uc3_533',
         'uc3_549',
         'un1_3180',
         'us3_new461'
        ]

In [ ]:
# make sure the samp name is legit
def fixsamp(s):
    if 'pije' in s:
        s = s.split("_pije")[0]
    if 'new' in s:
        while s not in t['converted'].tolist():
            plt_new,num = (s[:7],s[7:])
            s = "".join([plt_new,"0%s" % num])
    elif '0' in s:
        plot,num = s.split("_")
        if num.startswith("0"):
            while s not in t['converted'].tolist():
                num = num[1:]
                s = "_".join([plot,num])
    if s == 'uc2_9920':
        s = 'uc2_9220'
    if s == '691uc1_691':
        s = 'uc1_691'
    if 'blank' not in s:
        if s not in skips:
            assert s in t['converted'].tolist()
    return(s)

def doit(args):
    fmapp = OrderedDict()
    df, pos = args
    for row in df.index:
        samp = df.loc[row,1]
        splits = samp.split("_")
        if pos == 'first':
            locus = splits[0]
            if not 'blank' in splits:
                samp = "_".join([splits[1],splits[2]])
            else:
                samp = 'blank'
        if pos == 'last':
            locus = splits[-1]
            if not 'blank' in splits:
                samp = "_".join([splits[0],splits[1]])
            else:
                samp = 'blank'
        assert len(samp) > 2    
        if not samp in skips:
            samp = fixsamp(samp)
        fmapp[df.loc[row,0]] = samp #df.loc[row,0] is the well ID
    return(fmapp)

In [ ]:
# for each plate that doesn't have samp in filename, for each locus make a dict with k=well and v=samp
mapp = OrderedDict()
for f in mfiles:
    filE = op.basename(f).split(".txt")[0].split("_")
    plate = "_".join(filE[0:2])
    locus = filE[-1]
    #print plate, locus
    if not plate in mapp:
        mapp[plate] = OrderedDict()
    if not locus in mapp[plate]:
        mapp[plate][locus] = OrderedDict()
    df = pd.read_csv(f,sep='\t',header=None)
    args = [df]
    if f in locfirst:
        args.append('first')
        mapp[plate][locus] = doit(args)
    if f in loclast:
        args.append('last')
        mapp[plate][locus] = doit(args)        

In [ ]:
fixsamp('691uc1_691')

In [ ]:
fixsamp('bc1_new5')

In [ ]:
fixsamp('bc1_019')

<a id="geno calls"></a>
# get genotype calls for each sample

In [ ]:
# aug left samps eg 'leftover_01_A01'
# redo02_chloro and redo03_chloro could be excluded to minimize error calcs
DIR = '/home/lindb/teakettle/genemarker/outfiles/'
augleft = [f for f in fs(DIR) if 'aug' in f]
[augleft.append(f) for f in fs(DIR) if 'leftover' in f]
len(augleft)

In [ ]:
#these have 'samp' in the samp name, need slightly different samp ID attribution
#eg 'combo_02_samp_A01'
DIR = '/home/lindb/teakettle/genemarker/outfiles/'
comboredoplate7 = [f for f in fs(DIR) if 'combi' in f]
[comboredoplate7.append(f) for f in fs(DIR) if 'redo' in f]
[comboredoplate7.append(f) for f in fs(DIR) if 'plate' in f]
len(comboredoplate7)

In [ ]:
DIR = '/home/lindb/teakettle/genemarker/outfiles/'
files = fs(DIR)
len(files)

In [ ]:
nuc = ['rps39', 'rps12', 'rps2', 'rps50']

In [ ]:
#get the results from the shitty outfiles from GeneMarker (softgenics)
DIR = '/home/lindb/teakettle/genemarker/outfiles/'
files = fs(DIR)

regen = OrderedDict()
res = OrderedDict() # key = samp, val = key (locus) value = genotype
loci = ['pt71936','rps39','pc10','pt87268','rps12','rps2','rps50']
Fs = OrderedDict()
crap = OrderedDict()
gtcalls = OrderedDict() # key = file, value = key (samp) value = genotypes
for g in files:
    count =0
    locus = op.basename(g).split("_")[-1].split(".TXT")[0]
    if not g in gtcalls:
        gtcalls[g] = OrderedDict()

    with open(g,'rb') as f:
        
        df = f.readlines()[13:]
        for line in df:
            line = line.split("\t")
            if not line[0] == '':
                
                #have to figure out sample name
                if g in augleft:
                    plate = op.basename(g).split("_")
                    plate = "_".join(plate[0:2])
                    splits = line[1].split("_")
                    well = splits[2].split(".fsa")[0]
                    samp = mapp[plate][locus][well]
                elif g in comboredoplate7:
                    splits = line[1].split("_")
                    plate = "_".join(splits[0:2])
                    if 'combo' in plate:
                        plate = 'combined_%s' % splits[1]
                    well = splits[-1].split(".fsa")[0]
                    samp = mapp[plate][locus][well]
                else:
                    samp = "_".join(line[1].split("_")[0:2])
                    if '.fsa' in samp:
                        samp = samp.split(".fsa")[0]

                if 'blank' not in samp:
                    #correct the samp name if necessary
                    samp = fixsamp(samp)
                    
                    if samp not in res: #create the spotholders to be filled in
                        res[samp] = OrderedDict()
                        Fs[samp] = OrderedDict()
                        for l in loci:
                            res[samp][l] = []
                            Fs[samp][l] = []
#                     else:
#                         print "%s in res" % samp 
                    if samp not in regen:
                        regen[samp] = OrderedDict()
                        for l in loci:
                            regen[samp][l] = []
#                     else:
#                         print "%s in regen" % samp
                    
                    regen[samp][locus].append(g)
                    oldysamp = samp
                    try:
                        assert samp not in gtcalls[g]
                    except AssertionError:
                        print samp,locus,g
                    gtcalls[g][samp] = []

                    if len(res[samp][locus])>0: #possible regenotyping
#                         print samp,locus,res[samp][locus],Fs[samp][locus]
#                         print locus,line
#                         print op.basename(g)
#                         print '\n'
                        count +=1
                        if not 'dup' in samp:
                            samp = samp + '_dup'
                        scount = 0
                        while samp in res:
                            samp = samp + str(scount)
                            scount += 1
                        res[samp] = OrderedDict()
                        Fs[samp] = OrderedDict()
                        for l in loci:
                            res[samp][l] = []
                            Fs[samp][l] = []

                    for x in line[2:]:
                        if not x in ['\r\n','','\n']:
                            if '\r\n' in x: #not the same as: if not x in ['\r\n','','\n']
                                x = x.split('\r\n')[0]
                                print x,line
                            if samp == 'un3_1242':
#                                 print x
                                hhhh = 1
                            res[samp][locus].append(x)
                            gtcalls[g][oldysamp].append(x)
                            Fs[samp][locus].append(op.basename(g))
#             if samp == 'un3_1242':
#                 print locus,line, '\t', g

In [ ]:
plate,locus,well

In [ ]:
gtcalls[g][oldysamp]

In [ ]:
# how to get the well from the samp name
print mapp['aug_01']['pc10'].keys()[mapp['aug_01']['pc10'].values().index('uc3_123')]

<a id="print"></a>
# print out samps that have regenotyping data

In [ ]:
# for plates with files that include samp name, add these to mapp.dict
def mapit(p):
    splits = op.basename(p).split("_")
    platey = "_".join([splits[0],splits[1]]).replace("-","_")
    mappy = OrderedDict()
    with open(p,'rb') as f:
        df = f.readlines()[13:]
        for line in df:
            line = line.split("\t")
            if not line[0] == '':
                samp = "_".join(line[1].split("_")[0:2])        
                if '.fsa' in samp:
                    samp = samp.split(".fsa")[0]
                if not 'blank' in samp:
                    if not samp in skips:
                        samp = fixsamp(samp)
#                     well = convert[int(line[0])]
                    well = pm[platey][samp]
                    #assert well not in mapp[plate][locus]
                    samp = fixsamp(samp)
                    mappy[well] = samp
    return(mappy)

# #make a dictionary to convert the line number to the well number
# wells = []
# for letter in string.uppercase[0:8]:
#     for i in range(12):
#         j = i + 1
#         well = '%s%s' % (letter,str(j).zfill(2))
#         wells.append(well)
# convert = {}
# for i in range(96):
#     convert[i+1] = wells[i]
    
# get the file to convert samp to well ID
platemap = pd.read_csv('/home/lindb/teakettle/genemarker/plates3.csv',header=0,sep=',')
pm = {}
for col in platemap.columns[1:]:
    pm[col] = {}
    for row in platemap.index:
        samp = platemap.loc[row,col]
        samp = fixsamp(samp)
        pm[col][samp] = platemap.loc[row,'well']

In [ ]:
#print out all of the samples with regenotyping data
regenotyped = []
for samp in sorted(regen):
    for locus in sorted(loci):
        if locus in regen[samp].keys():
            if len(regen[samp][locus]) > 1:
                if samp not in regenotyped:
                    regenotyped.append(samp)
                print samp
                for f in regen[samp][locus]:
                    splits = op.basename(f).split("_")
                    plate = "_".join([splits[0],splits[1]])
                    if not plate in mapp: # these plates had the sample name as input
                        mapp[plate] = OrderedDict()
                    if not locus in mapp[plate]:
                        mapp[plate][locus] = mapit(f)
                    well = mapp[plate][locus].keys()[mapp[plate][locus].values().index(samp)]
                    calls = gtcalls[f][samp]
                    print well, calls, f
        else:
            print '%s not in regen[samp].keys()' % locus
            sys.exit()
print 'count = %s' % len(regenotyped) #count = 728

In [ ]:
#of the samples with regenotyping data, print out only those that have at least one called allele
regenotyped = []
for samp in sorted(regen):
    for locus in sorted(loci):
        if locus in regen[samp].keys():
            if len(regen[samp][locus]) > 1:
                if samp not in regenotyped:
                    regenotyped.append(samp)
                
                count = 0
                data = []
                for f in regen[samp][locus]:
                    splits = op.basename(f).split("_")
                    plate = "_".join([splits[0],splits[1]])
                    if not plate in mapp: # these plates had the sample name as input
                        mapp[plate] = OrderedDict()
                    if not locus in mapp[plate]:
                        mapp[plate][locus] = mapit(f)
                    well = mapp[plate][locus].keys()[mapp[plate][locus].values().index(samp)]
                    calls = gtcalls[f][samp]
                    data.append((well,calls,f))
                    
                    if len(calls) > 0:
                        count += 1
                if count > 0:
                    print samp
                    for d in data:
                        print d
        else:
            print '%s not in regen[samp].keys()' % locus
            sys.exit()
print 'count = %s' % len(regenotyped) #count = 728

In [ ]:
# for outfiles_corrected
#of the samples with regenotyping data, print out only those that have at least one called allele
regenotyped = []
for samp in sorted(regen):
    for locus in sorted(loci):
        if locus in regen[samp].keys():
            if len(regen[samp][locus]) > 1:
                if samp not in regenotyped:
                    regenotyped.append(samp)
                
                count = 0
                data = []
                for f in regen[samp][locus]:
                    splits = op.basename(f).split("_")
                    plate = "_".join([splits[0],splits[1]])
                    if not plate in mapp: # these plates had the sample name as input
                        mapp[plate] = OrderedDict()
                    if not locus in mapp[plate]:
                        mapp[plate][locus] = mapit(f)
                    well = mapp[plate][locus].keys()[mapp[plate][locus].values().index(samp)]
                    calls = gtcalls[f][samp]
                    data.append((well,calls,f))
                    
                    if len(calls) > 0:
                        count += 1
                if count > 0:
                    print samp
                    for d in data:
                        print d
        else:
            print '%s not in regen[samp].keys()' % locus
            sys.exit()
print 'count = %s' % len(regenotyped) #count = 728

<a id="calcs"></a>
# calc dropout, mistyping

In [ ]:
# save for easy upload later if notebook dies
rfilE = '/home/lindb/teakettle/genemarker/regengtpickles/regen_jan.pkl'
pkldump(regen,rfilE)
gfilE = '/home/lindb/teakettle/genemarker/regengtpickles/gtcalls_jan.pkl'
pkldump(gtcalls,gfilE)

In [ ]:
nuc = sorted(['rps39', 'rps12', 'rps2', 'rps50'])
nuc

In [ ]:
# round calls to integers
def irf(n):
    return int(round(float(n)))

# is genotype homozygous?
def homozy(C):
    if luni(C) == 1:
        return(True)
    else:
        return(False)

# calc dropout
def dropout(c):
    d = 0 # n dropouts (one for each dropout per locus per sample)
    t = 0 # total number of locus comparisons  (samples * loci)
    for i,icall in enumerate(c):
        c1 = icall.count('NA')
        h1 = homozy(icall)
        for j,jcall in enumerate(c):
            c2 = jcall.count('NA')
            h2 = homozy(jcall)
            
            if i < j: # n choose 2           
                t += 2
                if (c1,c2) != (2,2): # make sure both aren't NA (this may happen when len(c) > 2)
                    # if both have genotype calls and aren't blank
                    if c1 == 0 and c2 == 0:
                        # if one homozy and other het
                        if (h1,h2) == (True,False) or (h1,h2) == (False,True):
                            d += 1
                        # any other combo with both having genotype calls is due to genotyping error

                    # if one homo and other blank   
                    elif h1 and c2 == 2:
                        d += 2
                    elif h2 and c1 == 2:
                        d += 2

                    # if one het and other blank
                    elif c1 == 0 and c2 == 2:
                        d += 2
                    elif c2 == 0 and c1 == 2:
                        d += 2
    return (d,t)

# calc error rates based on threshold
def error(c,thresh): # thresh is adjustable to account for same geno within a few bps
# for program, want thresh = 0
# for curiosity, want: 0 <= thresh <= 4
    e = 0
    t = 0
    for i,icall in enumerate(c):
        c1 = icall.count('NA')
        h1 = homozy(icall)
        for j,jcall in enumerate(c):
            c2 = jcall.count('NA')
            h2 = homozy(jcall)
            
            if i < j: # n choose 2           
                if (c1,c2) != (2,2):        # make sure both aren't NA (this may happen when len(c) > 2)
                    if c1 == 0 and c2 == 0:     # if both have genotype calls; this is redundant to (c1,c2) != (2,2)
                        t += 2
                        if not icall == jcall:  # can skip those with same calls as these don't have errors
                            
                            # if one homozy and other het
                            if (h1,h2) == (True,False):    # if icall is homozy
                                if abs(icall[0] - jcall[0]) > thresh:
                                    e += 1
                                if abs(icall[0] - jcall[1]) > thresh:
                                    e += 1
                            elif (h1,h2) == (False,True):  # if jcall is homozy
                                if abs(jcall[0] - icall[0]) > thresh:
                                    e += 1
                                if abs(jcall[0] - icall[1]) > thresh:
                                    e += 1

                            # if both homozy - all chloro will fall into here
                            elif (h1,h2) == (True,True):
                                if abs(icall[0] - jcall[0]) > thresh: 
                                    e += 2

                            # if both het
#                             elif (h1,h2) == (False,False):
#                                 jmatched = ''
#                                 for i,J in enumerate(jcall):
#                                     matched = False
#                                     if not abs(icall[0] - J) > thresh:
#                                         matched = True
#                                         jmatched = J
#                                     if not matched:
#                                         if not abs(icall[1] - J) > thresh:
#                                             matched = True
#                                             jmatched = J
#                                     if not matched:
#                                         e += 1    
                                        
                            elif (h1,h2) == (False,False):
                                imatched = ''
                                for J in jcall:
                                    matched = False
                                    if not abs(icall[0] - J) > thresh:
                                        if not imatched == icall[0]:
                                            matched = True
                                            imatched = icall[0]
                                    if not matched:
                                        if not abs(icall[1] - J) > thresh:
                                            if not imatched == icall[1]:
                                                matched = True
                                                imatched = icall[1]
                                    if not matched:
                                        e += 1    
    return (e,t)

# get calls from a list of all genemarker outfiles used for (re)genotyping a specific sample
def getcalls(s,l):
    calls = []
    for g in sorted(regen[s][l]):
        call = gtcalls[g][s]
        assert type(call) == list
        assert len(call) <= 2
        if l in nuc:
            if len(call) == 0: # if blank use call twice
                calls.append(('NA','NA')) # call will not be in a list, will be a single string
            elif len(call) == 1:
                calls.append((irf(call[0]),irf(call[0])))
            else:
                calls.append((irf(call[0]),irf(call[1])))
        else: # if chloro marker
            if len(call) == 0:
                calls.append(('NA','NA'))
            else:
                calls.append((irf(call[0]),irf(call[0])))
    return (calls)

In [ ]:
error([(154,555),(155,154)],1), error([(155,154),(154,555)],1)

In [ ]:
samp = 'uc2_new004'
locus

In [ ]:
error([(154,555),(154,154)],0), error([(154,154),(154,555)],0)

In [ ]:
error([(154,555),(155,154)],1), error([(155,154),(154,555)],1)

In [ ]:
regen[samp][locus]

In [ ]:
gtcalls[regen[samp][locus][0]][samp]

In [ ]:
getcalls(samp,locus)

In [ ]:
error([[145,154],[165,154]],0)

In [ ]:
# test it out
dropout([('NA','NA'),('NA','NA')]), error([('NA','NA'),('NA','NA')],0)

In [ ]:
dropout([(154, 154),(154,153)]), error([(154, 154),(154,153)],0)

In [ ]:
dropout([('NA', 'NA'), ('NA', 'NA'),(154,154)]),error([('NA', 'NA'), ('NA', 'NA'),(154,154)],0)

In [ ]:
dropout([('NA', 'NA'), (165, 15),(154,154)]), error([('NA', 'NA'), (153,167),(165,154)],1)

In [ ]:
dropout([(143, 155), (137, 154)]), error([(143, 155), (137, 154)],1)

In [ ]:
# create dict to hold outputs for each locus
dropouts = {}
errors   = {}
for locus in loci:
    dropouts[locus] = {}
    errors[locus]   = {}
    for x in ['d','t']:
        dropouts[locus][x] = 0
        errors[locus][x] = 0

        
# determine dropout and error prob
for samp in sorted(regen): # loop through all samples; regen[samp][locus] is a list of files
    
    if samp in t['converted'].tolist(): # assert sample name
        for locus in sorted(loci):
#         for locus in ['pc10']:
#         for locus in ['rps39']:
            if locus in regen[samp].keys(): # if a samp has been genotyped for this locus
                
                # determine genotype calls across all files with samp genotyped for this locus
                calls = getcalls(samp,locus)
#                 for g in sorted(regen[samp][locus]): # this is now getcalls()
#                     call = gtcalls[g][samp]
#                     assert type(call) == list
#                     assert len(call) <= 2
#                     if locus in nuc:
#                         if len(call) == 0: # if blank use call twice
#                             calls.append(('NA','NA')) # call will not be in a list, will be a single string
#                         elif len(call) == 1:
#                             calls.append((irf(call[0]),irf(call[0])))
#                         else:
#                             calls.append((irf(call[0]),irf(call[1])))
#                     else: # if chloro marker
#                         if len(call) == 0:
#                             calls.append(('NA','NA'))
#                         else:
#                             calls.append((irf(call[0]),irf(call[0])))
                
                
                # how many calls aren't blank?
                full = len([call for call in calls if not call == ('NA','NA')])
                
                # determine dropout, error, and total number of comparisons
                do = 0
                tot = 0
                err = 0
                etot = 0
                if full > 0: # if calls aren't all blank
                    if len(calls) > 1: # can't do dropout/error calcs if samp genotyped only once
                        do,tot   = dropout(calls)
                        err,etot = error(calls,0)
#                         if err > 0:
#                             print samp,locus,calls,do,tot,err,etot
                dropouts[locus]['d'] += do
                dropouts[locus]['t'] += tot
                errors[locus]['d']   += err
                errors[locus]['t']   += etot
            
            else:
                print '%s has no locus %s' % (samp,locus)
    else:
        print "%s not in TFK" % samp

In [ ]:
error([(143, 155), (137, 154)],2)

In [ ]:
# look at raw dropouts and totals per locus
dropouts

In [ ]:
# look at dropout percentages
lst = []
for locus in dropouts: # if (c1,c2) != (2,2)
    lst.append([locus, dropouts[locus]['d']/dropouts[locus]['t']])
print tabulate(lst, headers=['locus','drop perc'])

In [ ]:
# look at raw errors and totals per locus - thresh = 0
errors

In [ ]:
# look at error percentages - thres = 0
lst = []
for locus in sorted(errors):
    lst.append([locus, errors[locus]['d']/errors[locus]['t']])
print tabulate(lst,headers=['locus','error perc'])

In [ ]:
# look at raw errors and totals per locus - thresh = 4
errors

In [ ]:
# look at error percentages - thres = 4
lst = []
for locus in sorted(errors):
    lst.append([locus, errors[locus]['d']/errors[locus]['t']])
print tabulate(lst,headers=['locus','error perc'])

<a id="infile"></a>
# Create infile to Moran & Clark

In [ ]:
# create a blank dict to use for final moran infile
# I want in specific order so I can print to file later
final = OrderedDict()
for samp in sorted(regen.keys()):
    if samp in t['converted'].tolist():
        if not samp in final:
            final[samp]           = OrderedDict()
            final[samp]['first']  = OrderedDict()
            final[samp]['second'] = OrderedDict()
        for locus in sorted(loci,reverse=True):
            if not locus in nuc:
                final[samp]['first'][locus]  = OrderedDict()
                final[samp]['second'][locus] = OrderedDict()
            else:
                final[samp]['first'][locus]  = OrderedDict()
                final[samp]['first']["%s_2"  % locus] = OrderedDict()
                final[samp]['second'][locus] = OrderedDict()
                final[samp]['second']["%s_2" % locus] = OrderedDict()
    else:
        print samp

In [ ]:
final.keys()[0]

In [ ]:
final[final.keys()[0]]

In [ ]:
# fill in final moran infile dict
clens = [] # so I don't have to write a bunch of exceptions
for samp in sorted(regen): # loop through all samples; regen[samp][locus] is a list of files
    if samp in t['converted'].tolist(): # assert sample name
        for locus in sorted(loci,reverse=True):
            if locus in regen[samp].keys(): # if a samp has been genotyped for this locus
                
                # determine genotype calls across all files with samp genotyped for this locus
                calls = getcalls(samp,locus)
                calls = [call for call in calls if not call == ('NA','NA')] # don't need blanks
                if len(calls) > 2:
                    print calls
                clens.append(len(calls))
                
                if len(calls) == 0:   # if all missing data
                    final[samp]['first'][locus]  = 0
                    final[samp]['second'][locus] = 0
                    if locus in nuc:
                        final[samp]['first']["%s_2" % locus]  = 0
                        final[samp]['second']["%s_2" % locus] = 0
                elif len(calls) == 1: # if only one good genotyping
                    final[samp]['first'][locus]  = calls[0][0]
                    final[samp]['second'][locus] = 0
                    if locus in nuc:
                        final[samp]['first']["%s_2" % locus]  = calls[0][1]
                        final[samp]['second']["%s_2" % locus] = 0
                elif len(calls) > 1: # for len(calls) {2,3}, all with len(calls)==3 are ok to use 1st & 2nd calls
                    final[samp]['first'][locus]  = calls[0][0]
                    final[samp]['second'][locus] = calls[1][0]
                    if locus in nuc:
                        final[samp]['first']["%s_2" % locus]  = calls[0][1]
                        final[samp]['second']["%s_2" % locus] = calls[1][1]
                
#                 # how many calls aren't blank?
#                 full = len([call for call in calls if not call == ('NA','NA')])
                
#                 # fill in dict(final)
#                 if full > 0: # if calls aren't all blank
#                     if len(calls) > 1:
#                         cs = [call for call in calls if not call == ('NA','NA')]
#                         assert len(cs) < 4
#                         clens.append(len(cs))
                        
#                     else: #if only one call; can't be zero calls because full would = 0 and else would take effect
#                         final[samp]['first'][locus] = calls[0][0]
#                         final[samp]['second'][locus]= 0
#                         if locus in nuc:
#                             final[samp]['first']['%s_2' % locus] = calls[0][1]
#                             final[samp]['second']['%s_2' % locus]= 0
#                     if len(calls) == 0:
#                         print 'crap'
                        
                        
#                 else: # fill all as blank
#                     final[samp]['first'][locus]  = 0
#                     final[samp]['second'][locus] = 0
#                     if locus in nuc:
#                         final[samp]['first']["%s_2" % locus]  = 0
#                         final[samp]['second']["%s_2" % locus] = 0

            
            else:
                print '%s has no locus %s' % (samp,locus)
    else:
        print "%s not in TFK" % samp

In [ ]:
table(clens)

# print results to table

In [ ]:
final[final.keys()[0]]['first'].keys()

In [ ]:
final[final.keys()[0]]['first'].values()

In [ ]:
final[final.keys()[0]]['first']

In [ ]:
sorted(loci,reverse=True)

In [ ]:
#write out the results to a df
nuc = ['rps39','rps12','rps2','rps50']
cols = ['rps50',
        'rps50_2',
        'rps39',
        'rps39_2',
        'rps2',
        'rps2_2',
        'rps12',
        'rps12_2',
        'pt87268',
        'pt71936',
        'pc10']
filE = '/home/lindb/teakettle/genemarker/results_after_jan.tsv'
with open(filE,'wb') as o:
    text = 'samp' + '\t' + '\t'.join([l for l in cols]) + '\n'
    o.write("%s" % text)
    for samp in sorted(final):
        for typing in final[samp]:
            text = samp + '\t' + '\t'.join([str(val) for val in final[samp][typing].values()]) +'\n'
    #         print text
    #         print samp
            o.write("%s" % text)

In [ ]:
genotypes = pd.read_csv('/home/lindb/teakettle/genemarker/results_after_jan.tsv',sep='\t')
genotypes.replace(np.nan,"NA",inplace=True)
genotypes.head()

In [ ]:
'691uc1_691' in genotypes['samp'].tolist()

In [ ]:
'uc2_9920' in genotypes['samp'].tolist()

In [ ]:
'uc1_691' in genotypes['samp'].tolist()

In [ ]:
genotypes[genotypes.samp.str.contains('bc1_1216')]

In [ ]:
nrow(genotypes) == 2*luni(genotypes['samp'])

In [ ]:
regen['bc1_1179']